In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
# Load datasets
df1 = pd.read_csv('/home/wahba/Documents/cicid/cicids2017/original/csv/TrafficLabelling /Wednesday-workingHours.pcap_ISCX.csv')
df2 = pd.read_csv('/home/wahba/Documents/cicid/CICFlowMeter/data/daily/2025-11-22_Flow.csv')
df3 = pd.read_csv('/home/wahba/Documents/nids3/tests/flow/flow_wednesday_flow_generation7.csv')

In [4]:
# Cleaning column names
col_names = {col: col.strip() for col in df1.columns}
df1.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df2.columns}
df2.rename(columns=col_names, inplace=True)
col_names = {col: col.strip() for col in df3.columns}
df3.rename(columns=col_names, inplace=True)

del col_names

In [5]:
df1['Label'].value_counts()

Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64

In [6]:
# Filter out only DoS slowhttptest attacks
df1_dos = df1[df1['Label'] == 'DoS Slowhttptest']

In [7]:
df1_dos['flow_key'] = (
    df1_dos['Source IP'].astype(str) + ':' +
    df1_dos['Source Port'].astype(str) + '-' +
    df1_dos['Destination IP'].astype(str) + ':' +
    df1_dos['Destination Port'].astype(str) + '-' +
    df1_dos['Protocol'].astype(str)
    )
df2['flow_key'] = (
    df2['Src IP'].astype(str) + ':' +
    df2['Src Port'].astype(str) + '-' +
    df2['Dst IP'].astype(str) + ':' +
    df2['Dst Port'].astype(str) + '-' +
    df2['Protocol'].astype(str)
    )
df3['Protocol'] = df3['Protocol'].replace({'TCP': 6, 'UDP': 17, 'ICMP': 1}).astype(int)
df3['flow_key'] = (
    df3['Source IP'].astype(str) + ':' +
    df3['Source Port'].astype(str) + '-' +
    df3['Destination IP'].astype(str) + ':' +
    df3['Destination Port'].astype(str) + '-' +
    df3['Protocol'].astype(str)
    )

/tmp/ipykernel_207510/2829646784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_dos['flow_key'] = (
/tmp/ipykernel_207510/2829646784.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df3['Protocol'] = df3['Protocol'].replace({'TCP': 6, 'UDP': 17, 'ICMP': 1}).astype(int)


In [8]:
# Get unique flow keys for slowhttptest from df1

slowhttptest_flow_key = df1_dos['flow_key'].unique()

In [9]:
# Filter df2 and df3 for flows matching slowhttptest flow keys
df2_dos = df2[df2['flow_key'].isin(slowhttptest_flow_key)].reset_index(drop=True)
df3_dos = df3[df3['flow_key'].isin(slowhttptest_flow_key)].reset_index(drop=True)

# Find flow keys appeared in both dataset
df2_dos_and_df3_dos_matched_flow = set(df2_dos['flow_key']) & set(df3_dos['flow_key'])

In [10]:
# derive new df comprising flow_key appeared on both df
df2_dos_matched_df3 = df2_dos[df2_dos['flow_key'].isin(df2_dos_and_df3_dos_matched_flow)]
df3_dos_matched_df2 = df3_dos[df3_dos['flow_key'].isin(df2_dos_and_df3_dos_matched_flow)] 

# Comparing CICFLOWMETER captured flows with custom python generated flow

In [18]:
# Comparing shapes
print("df2_dos.shape: ", df2_dos.shape)
print("df2_dos_matched_df3.shape: ",df2_dos_matched_df3.shape)
print("df3_dos.shape: ", df3_dos.shape)
print("df2_dos_matched_df3.shape:", df3_dos_matched_df2.shape)

df2_dos.shape:  (732, 85)
df2_dos_matched_df3.shape:  (361, 85)
df3_dos.shape:  (379, 58)
df2_dos_matched_df3.shape: (207, 58)


In [19]:
print(df2_dos_and_df3_dos_matched_flow)

{'172.16.0.1:33442-192.168.10.50:80-6', '172.16.0.1:35510-192.168.10.50:80-6', '172.16.0.1:33560-192.168.10.50:80-6', '172.16.0.1:33412-192.168.10.50:80-6', '172.16.0.1:33516-192.168.10.50:80-6', '172.16.0.1:33566-192.168.10.50:80-6', '172.16.0.1:33572-192.168.10.50:80-6', '172.16.0.1:33434-192.168.10.50:80-6', '172.16.0.1:33426-192.168.10.50:80-6', '172.16.0.1:33616-192.168.10.50:80-6', '172.16.0.1:33374-192.168.10.50:80-6', '172.16.0.1:33602-192.168.10.50:80-6', '172.16.0.1:35494-192.168.10.50:80-6', '172.16.0.1:33438-192.168.10.50:80-6', '172.16.0.1:33514-192.168.10.50:80-6', '172.16.0.1:35490-192.168.10.50:80-6', '172.16.0.1:33462-192.168.10.50:80-6', '172.16.0.1:33502-192.168.10.50:80-6', '172.16.0.1:33540-192.168.10.50:80-6', '172.16.0.1:34880-192.168.10.50:80-6', '172.16.0.1:33642-192.168.10.50:80-6', '172.16.0.1:33378-192.168.10.50:80-6', '172.16.0.1:34026-192.168.10.50:80-6', '172.16.0.1:35858-192.168.10.50:80-6', '172.16.0.1:33510-192.168.10.50:80-6', '172.16.0.1:33480-192.16

In [20]:
flow_key = '172.16.0.1:34436-192.168.10.50:80-6'

print(df2_dos[df2_dos['flow_key'] == flow_key].index)
print(df3_dos[df3_dos['flow_key'] == flow_key].index)

Index([81, 83], dtype='int64')
Index([105], dtype='int64')


In [21]:
print(df2_dos.columns)
print(df3_dos.columns)

Index(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol',
       'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Len

In [22]:
print(df2_dos.loc[[81, 83], ['Timestamp', 'Flow Duration']])
print(df3_dos.loc[[105], ['Start Time', 'Flow Duration']])

                 Timestamp  Flow Duration
81  22/11/2025 06:04:55 PM      116729922
83  22/11/2025 06:07:45 PM            107
       Start Time  Flow Duration
105  1.763806e+09  169484.172106


# Test model prediction on flow generated by CICFLOWMETER and from CICIDS2017

In [23]:
import joblib

rf = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
xgb = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
knn = joblib.load("/home/wahba/Documents/models_anacletu/knn_model.joblib")
robust_scaler = joblib.load("/home/wahba/Documents/models_anacletu/robust_scaler.joblib")

/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
/home/wahba/miniconda3/envs/ml2/lib/python3.12/pickle.py:1760: UserWarning: [19:58:43] WARNING: /croot/xgboost-split_1749630910898/work/src/collective/../data/../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that 

# Standardizing dataframes to feed to ML model

In [24]:
# Standardizing column names

final_columns_df1 = {
    'Destination Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packets': 'Total Fwd Packets',
    'Total Length of Fwd Packets': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Min Packet Length': 'Min Packet Length',
    'Max Packet Length': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'Init_Win_bytes_forward': 'Init_Win_bytes_forward',
    'Init_Win_bytes_backward': 'Init_Win_bytes_backward',
    'act_data_pkt_fwd': 'act_data_pkt_fwd',
    'min_seg_size_forward': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

final_columns_df2 = {
    'Dst Port': 'Destination Port',
    'Flow Duration': 'Flow Duration',
    'Total Fwd Packet': 'Total Fwd Packets',
    'Total Length of Fwd Packet': 'Total Length of Fwd Packets',
    'Fwd Packet Length Max': 'Fwd Packet Length Max',
    'Fwd Packet Length Min': 'Fwd Packet Length Min',
    'Fwd Packet Length Mean': 'Fwd Packet Length Mean',
    'Fwd Packet Length Std': 'Fwd Packet Length Std',
    'Bwd Packet Length Max': 'Bwd Packet Length Max',
    'Bwd Packet Length Min': 'Bwd Packet Length Min',
    'Bwd Packet Length Mean': 'Bwd Packet Length Mean',
    'Bwd Packet Length Std': 'Bwd Packet Length Std',
    'Flow Bytes/s': 'Flow Bytes/s',
    'Flow Packets/s': 'Flow Packets/s',
    'Flow IAT Mean': 'Flow IAT Mean',
    'Flow IAT Std': 'Flow IAT Std',
    'Flow IAT Max': 'Flow IAT Max',
    'Flow IAT Min': 'Flow IAT Min',
    'Fwd IAT Total': 'Fwd IAT Total',
    'Fwd IAT Mean': 'Fwd IAT Mean',
    'Fwd IAT Std': 'Fwd IAT Std',
    'Fwd IAT Max': 'Fwd IAT Max',
    'Fwd IAT Min': 'Fwd IAT Min',
    'Bwd IAT Total': 'Bwd IAT Total',
    'Bwd IAT Mean': 'Bwd IAT Mean',
    'Bwd IAT Std': 'Bwd IAT Std',
    'Bwd IAT Max': 'Bwd IAT Max',
    'Bwd IAT Min': 'Bwd IAT Min',
    'Fwd Header Length': 'Fwd Header Length',
    'Bwd Header Length': 'Bwd Header Length',
    'Fwd Packets/s': 'Fwd Packets/s',
    'Bwd Packets/s': 'Bwd Packets/s',
    'Packet Length Min': 'Min Packet Length',
    'Packet Length Max': 'Max Packet Length',
    'Packet Length Mean': 'Packet Length Mean',
    'Packet Length Std': 'Packet Length Std',
    'Packet Length Variance': 'Packet Length Variance',
    'FIN Flag Count': 'FIN Flag Count',
    'PSH Flag Count': 'PSH Flag Count',
    'ACK Flag Count': 'ACK Flag Count',
    'Average Packet Size': 'Average Packet Size',
    'Subflow Fwd Bytes': 'Subflow Fwd Bytes',
    'FWD Init Win Bytes': 'Init_Win_bytes_forward',
    'Bwd Init Win Bytes': 'Init_Win_bytes_backward',
    'Fwd Act Data Pkts': 'act_data_pkt_fwd',
    'Fwd Seg Size Min': 'min_seg_size_forward',
    'Active Mean': 'Active Mean',
    'Active Max': 'Active Max',
    'Active Min': 'Active Min',
    'Idle Mean': 'Idle Mean',
    'Idle Max': 'Idle Max',
    'Idle Min': 'Idle Min'
}

In [25]:
df1_dos_standardized = df1_dos[list(final_columns_df1.keys())].rename(columns=final_columns_df1)
df2_dos_standardized = df2_dos[list(final_columns_df2.keys())].rename(columns=final_columns_df2)
df3_dos_standardized = df3_dos[list(final_columns_df1.keys())].rename(columns=final_columns_df1)
print("expected number of features for the model:", len(final_columns_df1))

expected number of features for the model: 52


In [26]:
print("df1_dos_standardized shape:", df1_dos_standardized.shape)
print("df2_dos_standardized shape:",df2_dos_standardized.shape)
print("df3_dos_standardized shape:", df3_dos_standardized.shape)
print("Expected number of features for the model:", len(final_columns_df1))

df1_dos_standardized shape: (5499, 52)
df2_dos_standardized shape: (732, 52)
df3_dos_standardized shape: (379, 52)
Expected number of features for the model: 52


In [27]:
import joblib

rf = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
xgb = joblib.load("/home/wahba/Documents/models_anacletu/xgboost.joblib")
knn = joblib.load("/home/wahba/Documents/models_anacletu/knn_model.joblib")
robust_scaler = joblib.load("/home/wahba/Documents/models_anacletu/robust_scaler.joblib")

/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/wahba/miniconda3/envs/ml2/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [28]:
print(rf.classes_)
print(xgb.classes_)
print(knn.classes_)

[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]
['Bots' 'Brute Force' 'DDoS' 'DoS' 'Normal Traffic' 'Port Scanning'
 'Web Attacks']


In [29]:
rf_predictions_df1 = pd.Series(rf.predict(df1_dos_standardized))
xgb_predictions_df1 = pd.Series(xgb.predict(df1_dos_standardized))
knn_predictions_df1 = pd.Series(knn.predict(robust_scaler.transform(df1_dos_standardized)))

rf_predictions_df2 = pd.Series(rf.predict(df2_dos_standardized))
xgb_predictions_df2 = pd.Series(xgb.predict(df2_dos_standardized))
knn_predictions_df2 = pd.Series(knn.predict(robust_scaler.transform(df2_dos_standardized)))

rf_predictions_df3 = pd.Series(rf.predict(df3_dos_standardized))
xgb_predictions_df3 = pd.Series(xgb.predict(df3_dos_standardized))
knn_predictions_df3 = pd.Series(knn.predict(robust_scaler.transform(df3_dos_standardized)))

In [34]:
print("rf preditions df1\n",rf_predictions_df1.value_counts(), "\n")
print("xgb preditions df1\n",xgb_predictions_df1.value_counts(), "\n")
print("knn preditions df1\n",knn_predictions_df1.value_counts(), "\n")
print("rf preditions df2\n",rf_predictions_df2.value_counts(), "\n")
print("xgb preditions df2\n",xgb_predictions_df2.value_counts(), "\n")
print("knn preditions df2\n",knn_predictions_df2.value_counts(), "\n")
print("rf preditions df3\n",rf_predictions_df3.value_counts(), "\n")
print("xgb preditions df3\n",xgb_predictions_df3.value_counts(), "\n")
print("knn preditions df3\n",knn_predictions_df3.value_counts(), "\n")

rf preditions df1
 1    5493
0       4
3       1
5       1
Name: count, dtype: int64 

xgb preditions df1
 1    5493
0       4
3       1
5       1
Name: count, dtype: int64 

knn preditions df1
 DoS               5308
Normal Traffic     189
Web Attacks          1
DDoS                 1
Name: count, dtype: int64 

rf preditions df2
 1    731
0      1
Name: count, dtype: int64 

xgb preditions df2
 1    731
0      1
Name: count, dtype: int64 

knn preditions df2
 DoS               439
Normal Traffic    291
DDoS                2
Name: count, dtype: int64 

rf preditions df3
 1    361
0     18
Name: count, dtype: int64 

xgb preditions df3
 1    361
0     18
Name: count, dtype: int64 

knn preditions df3
 DoS               251
Normal Traffic    125
Port Scanning       3
Name: count, dtype: int64 

